In [1]:
import numpy as np 
import pandas as pd
from src import utils
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
def datetime64_to_matlab_serial(dates):
    """
    Convert an array of datetime64 values to MATLAB serial date numbers.

    Parameters:
    dates (numpy.ndarray): Array of datetime64 values.

    Returns:
    list: List of MATLAB serial date numbers.
    """
    matlab_reference_date = datetime(1970, 1, 1)  # Unix epoch

    # Convert datetime64 to Python datetime
    python_datetimes = [pd.Timestamp(dt).to_pydatetime() for dt in dates]

    # Calculate the MATLAB serial date number
    matlab_serial_dates = []
    for dt in python_datetimes:
        delta = dt - matlab_reference_date
        matlab_serial_date = 719529 + delta.days + delta.seconds / 86400.0 + delta.microseconds / 86400.0 / 1e6
        matlab_serial_dates.append(matlab_serial_date)

    return np.array(matlab_serial_dates)

def datetime2datenum(dt):
    """ dt: list or array, y:m:d:h:m:s:ms """
    import datetime
    dtime = datetime.datetime(dt[0], dt[1], dt[2], dt[3], dt[4], dt[5], dt[6])
    mdn = dtime + datetime.timedelta(days = 366)
    frac_seconds = (dtime-datetime.datetime(dtime.year,dtime.month,dtime.day,0,0,0)).seconds / (24.0 * 60.0 * 60.0)
    frac_microseconds = dtime.microsecond / (24.0 * 60.0 * 60.0 * 1000000.0)
    return mdn.toordinal() + frac_seconds + frac_microseconds

def get_pupil_df(m1, pupil_path: str = "Z:\data\Miguel\Facevideos"):
    from pathlib import Path
    pupil_path = Path(pupil_path)
    p_path = Path.joinpath(pupil_path, m1.name, m1.datexp, m1.blk, f"{m1.name}_{m1.datexp}_{m1.blk}_pupil.npy")
    pupil = np.load(p_path, allow_pickle=True)
    df = pd.DataFrame(pupil, columns=['Y', 'M', 'D', 'H', 'MM', 'S', 'MS', 'pupil_x', 'pupil_y', 'area'])
    df['time'] = df.apply(lambda x: datetime(x['Y'].astype(int), x['M'].astype(int), x['D'].astype(int), 
                                            x['H'].astype(int), x['MM'].astype(int), x['S'].astype(int), x['MS'].astype(int)), axis=1)
    df.drop(columns=['Y', 'M', 'D', 'H', 'MM', 'S', 'MS'], inplace=True)
    df['serialtime'] = df.apply(lambda x: datetime64_to_matlab_serial(np.array([x['time']]))[0], axis=1)
    return df

def get_frametimes(MouseObject):
    from scipy.interpolate import interp1d
    frames_time = MouseObject._data_var[0]
    ixx = (frames_time[:-1] > 2.5) * (frames_time[1:] < 2.5)
    iframes = np.argwhere(ixx).flatten()
    isamp = np.argwhere(MouseObject._data_var[1] > 1).flatten()
    ts = MouseObject._data_var[1][isamp]
    tframes = interp1d(isamp, ts)(iframes)
    nframes = len(tframes)
    return tframes, nframes

def load_cam_features(m1, camera_path: str = "Z:\data\Miguel\Facevideos"):
    from pathlib import Path
    camera_path = Path(camera_path)
    cam_path = Path.joinpath(camera_path, m1.name, m1.datexp, m1.blk, f"{m1.name}_{m1.datexp}_{m1.blk}_full_proc.npy")
    cam_proc = np.load(cam_path, allow_pickle=True).item()
    return cam_proc

def interp_to_corridor(frameselector, feature: str):
    ntrials = frameselector["trial_no"].astype(int).max()
    interp_feature = np.empty((ntrials, 400))
    for trial_no in frameselector["trial_no"].unique():
        trial_no = int(trial_no)
        trial_feature= frameselector.query(f"trial_no == {trial_no}")[feature].values
        trial_distance = frameselector.query(f"trial_no == {trial_no}")["distance"].values
        if (trial_distance[0] > 0) & (trial_no > 1):
            previous_trial_feature= frameselector.query(f"trial_no == {trial_no-1}")[feature].values[-1]
            #append 0 to the beginning
            trial_distance = np.insert(trial_distance, 0, 0)
            trial_feature = np.insert(trial_feature, 0, previous_trial_feature)
        elif (trial_distance[0] > 0) & (trial_no == 1):
            trial_distance = np.insert(trial_distance, 0, 0)
            trial_feature = np.insert(trial_feature, 0, trial_feature[0])
        trial_distance = np.round(trial_distance, 0).astype(int)
        interp_feature[trial_no-1] = np.interp(np.arange(400), trial_distance, trial_feature)
    return interp_feature

def plot_feature(feature, ylabel: str, labelypos: int = 60, ylabel_offset: int = 0):
    from scipy.stats import sem
    cat_a = feature[:,[0,2],:].mean(1)
    cat_b = feature[:,[1,3],:].mean(1)
    rew_gran_mean = cat_a.mean(axis=0)
    rew_gran_sem = sem(cat_a, axis=0)
    nrew_gran_mean = cat_b.mean(axis=0)
    nrew_gran_sem = sem(cat_b, axis=0)
    fig, ax = plt.subplots(1,2, figsize=(8, 4), sharex=True, sharey=True)
    ax[0].plot(rew_gran_mean[0], color='tab:green')
    ax[0].fill_between(np.arange(400), rew_gran_mean[0]-rew_gran_sem[0], rew_gran_mean[0]+rew_gran_sem[0], alpha=0.3, color='tab:green')
    ax[0].plot(nrew_gran_mean[0], color='tab:red')
    ax[0].fill_between(np.arange(400), nrew_gran_mean[0]-nrew_gran_sem[0], nrew_gran_mean[0]+nrew_gran_sem[0], alpha=0.3, color='tab:red')
    ax[1].plot(rew_gran_mean[1],  color='tab:green')
    ax[1].fill_between(np.arange(400), rew_gran_mean[1]-rew_gran_sem[1], rew_gran_mean[1]+rew_gran_sem[1], alpha=0.3, color='tab:green')
    ax[1].plot(nrew_gran_mean[1], color='tab:red')
    ax[1].fill_between(np.arange(400), nrew_gran_mean[1]-nrew_gran_sem[1], nrew_gran_mean[1]+nrew_gran_sem[1], alpha=0.3, color='tab:red')
    ax[0].set_ylabel(ylabel)
    ax[0].text(10,labelypos + ylabel_offset, "Active reward collection trials", color='tab:green')
    ax[0].text(10,labelypos, "Non rewarded trials with licks", color='tab:red')
    ax[1].text(10,labelypos + ylabel_offset, "Ignored rewarded trials", color='tab:green')
    ax[1].text(10,labelypos, "Correctly ommited non rewarded trials", color='tab:red')
    for i in range(2):
        ax[i].set_xlabel("Position (cm)")
        ax[i].axvline(150, color='k', linestyle='--', alpha=0.5)
        ax[i].axvline(250, color='k', linestyle='--', alpha=0.5)
        ax[i].axvline(300, color='k', linestyle='--', alpha=0.5)

def get_interp_speed(m1):
    #compute speed
    df = utils.get_movement_df(m1)
    ntrials = m1.frameselector["trial_no"].astype(int).max()
    speed_interp = np.empty((ntrials, 400))
    #interp speed to 400 positions
    for trial_no in m1.frameselector["trial_no"].unique():
        trial_no = int(trial_no)
        trial_speed = df.query(f"trial == {trial_no}")["pitch"].values * 10
        trial_distance = df.query(f"trial == {trial_no}")["distance"].values
        speed_interp[trial_no-1] = np.interp(np.arange(400), trial_distance, trial_speed)
    return speed_interp

In [3]:
from matplotlib import rcParams
default_font = 12
fs_title = 16
rcParams["font.family"] = "Arial"
rcParams["savefig.dpi"] = 300
rcParams["axes.spines.top"] = False
rcParams["axes.spines.right"] = False
rcParams["axes.titlelocation"] = "left"
rcParams["axes.titleweight"] = "normal"
rcParams["font.size"] = default_font
trial_type_palette = ['tab:green', 'tab:red', 'tab:cyan', 'tab:orange']

In [19]:
db = []
db.append({'mname': 'VG21', 'datexp': '2025_07_17', 'blk':'3'})
db.append({'mname': 'VG24', 'datexp': '2025_07_10', 'blk':'2'})
db

[{'mname': 'VG21', 'datexp': '2025_07_17', 'blk': '3'},
 {'mname': 'VG24', 'datexp': '2025_07_10', 'blk': '2'}]

In [22]:
from scipy.interpolate import interp1d
from pathlib import Path
svds_path = Path("Z:\data\Miguel\Facevideos")
for iss, sess in enumerate(db):
    name = sess['mname']
    date = sess['datexp']
    blk = sess['blk']
    m1 = utils.load_mouse(name, date, blk, load_neurons=True, interp_behav=True, load_retinotopy=True, mdl_path=r"D:\mouseobj")
    cam_proc = load_cam_features(m1, svds_path)
    motSVD = cam_proc['motSVD'][1]
    movSVD = cam_proc['movSVD'][1]
    motion_energy = cam_proc['motion'][1]
    pupil_df = get_pupil_df(m1, svds_path)
    tframes, nframes = get_frametimes(m1)
    camera_time = pupil_df['serialtime'].values
    interp_movSVD0 = interp1d(camera_time, movSVD[:,0], fill_value="extrapolate")(tframes)
    interp_motSVD0 = interp1d(camera_time, motSVD[:,0], fill_value="extrapolate")(tframes)
    interp_pupil_x = interp1d(camera_time, pupil_df['pupil_x'].values, fill_value="extrapolate")(tframes)
    interp_pupil_y = interp1d(camera_time, pupil_df['pupil_y'].values, fill_value="extrapolate")(tframes)
    interp_area = interp1d(camera_time, pupil_df['area'].values, fill_value="extrapolate")(tframes)
    interp_motion = interp1d(camera_time, motion_energy, fill_value="extrapolate")(tframes)
    cam_df = pd.DataFrame({'movSVD0': interp_movSVD0, 'motSVD0': interp_motSVD0, 
                        'pupil_x': interp_pupil_x, 'pupil_y': interp_pupil_y, 
                        'pupil_area': interp_area, 'motion_energy': interp_motion})
    fselector = pd.merge(m1.frameselector, cam_df, left_index=True, right_index=True)
    interp_movSVD0 = interp_to_corridor(fselector, 'movSVD0')
    interp_motSVD0 = interp_to_corridor(fselector, 'motSVD0')
    interp_pupil_x = interp_to_corridor(fselector, 'pupil_x')
    interp_pupil_y = interp_to_corridor(fselector, 'pupil_y')
    interp_area = interp_to_corridor(fselector, 'pupil_area')
    interp_motion = interp_to_corridor(fselector, 'motion_energy')
    speed_interp = get_interp_speed(m1)
    _, lick_rate = utils.get_lick_rates(m1)
    features = [interp_movSVD0, interp_motSVD0, interp_pupil_x, interp_pupil_y, interp_area, interp_motion]
    f_names = ['movSVD0', 'motSVD0', 'pupil_x', 'pupil_y', 'pupil_area', 'motion_energy']
    for i in range(len(features)):
        pth = Path(f"../data/{m1.name}/{m1.datexp}/{m1.blk}")
        if not pth.exists():
            pth.mkdir(parents=True, exist_ok=True)
        np.save(pth/f"{f_names[i]}_corridor.npy", features[i])
    np.save(pth/"speed_interp.npy", speed_interp)
    np.save(pth/"lick_rate.npy", lick_rate)

Checking if model object exists ...
Loading mouse object from D:\mouseobj\VG21\2025_07_17\3
Existing mouse object has the following attributes:
dict_keys(['name', 'datexp', 'blk', 'data_path', '_timeline', '_data_var', '_settings', '_timestamps', '_trial_info', '_spks', '_ypos', '_xpos', '_iplane', '_stat', '_ops', '_snr', 'xy_t', 'iarea', 'iregion', 'outline', 'frameselector', 'isred', 'trial_dict', 'interp_spks', 'train_dp'])
Checking if model object exists ...
Loading mouse object from D:\mouseobj\VG24\2025_07_10\2
Existing mouse object has the following attributes:
dict_keys(['name', 'datexp', 'blk', 'data_path', '_timeline', '_data_var', '_settings', '_timestamps', '_trial_info', '_spks', '_ypos', '_xpos', '_iplane', '_stat', '_ops', '_snr', 'xy_t', 'iarea', 'iregion', 'outline', 'frameselector', 'isred', 'trial_dict', 'interp_spks', 'train_dp'])
